# Action detection by 3DCNN

### 0, Setup

if you are not install these module, you must write in following these command on PowerShell or Cmd

```
pip install -q imageio
pip install -q opencv-python
pip install -q git+https://github.com/tensorflow/docs
```

In [1]:
# Import Module

from absl import logging
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np
import imageio
from IPython import display
from urllib import request

### 1, Create helper function for UCF101 dataset 

In [3]:
UCF_ROOT = 'https://www.crcv.ucf.edu/THUMOS14/UCF101/'
_VIDEO_LIST = None
_CACHE_DIR = tempfile.mkdtemp()

unverified_context = ssl._create_unverified_context()

# ucfのビデオデータセットを読み込んでリストとして格納する
def list_ucf_videos():
    """LIsts videos available in UCF101 dataset"""
    global _VIDEO_LIST
    if not _VIDEO_LIST:
        index = request.urlopen(UCF_ROOT, context=unverified_context).read().decode("utf-8")
        videos = re.findall("(v_[\w_]+\.avi", index)
        _VIDEO_LIST = sorted(set(videos))
    return list(_VIDEO_LIST)

# ビデオデータをフェッチキャッシュしてローカルファイルに保存する
def fetch_ucf_ideo(video):
    """Fetchs a video and cache into local filesystem"""
    cache_path = os.path.join(_CACHE_DIR, video)
    if not os.path.exists(cache_path):
        urlpath = request.urljoin(UCF_ROOT, video)
        print(f"Fetching {urlpath} => {cache_path}")
        data = request.urlopen(urlpath, context=unverified_context).read()
        open(cache_path, "wb").write(data)
    return cache_path

# フレームの中心から一定の距離離れている部分までを正方形に切り取る
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y:start_y+min_dim, start_x:start_x+min_dim]

# 一フレームずつごとに切り出して1~0までの行列に画像を変換させる
def load_video(path, max_frames=0, resize=(224, 224)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames) / 255.0

# 画像データをgit形式にして保存する
def to_gif(images):
    converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
    imageio.mimsave('./animation.gif', converted_images, fps=25)
    return embed.embed_file('./animation.gif')



In [ ]:
# Get the kinetics-400 action labels from the GitHub repository.
KINETICS_URL = ""